In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers,models

# Model

In [ ]:
vgg16_model = VGG16(weights = 'imagenet',input_shape=(224,224,3))
vgg16_model.summary()

553467904/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [ ]:
model = models.Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [ ]:
for layer in model.layers:
  layer.trainable = False


In [ ]:
model.add(layers.Dense(units=2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [ ]:
model.compile(loss ='categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])

# Data access from folder

In [ ]:
train_data_dir = "/content/drive/MyDrive/training_set/training_set"

In [ ]:
test_data_dir = "/content/drive/MyDrive/test_set/test_set"

## Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [ ]:
print('Total number of images for "training":')
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (224,224), 
batch_size = 16,
class_mode = 'categorical')

Total number of images for "training":
Found 8005 images belonging to 2 classes.


In [ ]:
print('Total number of images for "testing":')
test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size = (224,224), 
batch_size = 16,
class_mode = 'categorical')

Total number of images for "testing":
Found 2023 images belonging to 2 classes.


# Model Training

In [ ]:
model.fit(train_generator,epochs=5,steps_per_epoch=len(train_generator),verbose=2)

Epoch 1/5
501/501 - 2939s - loss: 0.1040 - accuracy: 0.9720
Epoch 2/5
501/501 - 40s - loss: 0.0522 - accuracy: 0.9860
Epoch 3/5
501/501 - 40s - loss: 0.0248 - accuracy: 0.9910
Epoch 4/5
501/501 - 40s - loss: 0.0141 - accuracy: 0.9948
Epoch 5/5
501/501 - 40s - loss: 0.0160 - accuracy: 0.9958


# Evaluation

In [19]:
Y_pred = model.predict(test_generator,steps=len(test_generator),verbose = 0)

In [20]:
y_pred = np.argmax(Y_pred ,axis=1)

In [21]:
y_pred

array([1, 0, 0, ..., 1, 0, 1])

array([1, 0, 0, ..., 1, 1, 0])

In [22]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy = accuracy_score(test_generator.classes, y_pred)
print("Accuracy in test set: %0.1f%% " % (accuracy * 100))

Accuracy in test set: 49.2% 
Accuracy in test set: 50.9% 


In [23]:
confusion_matrix(test_generator.classes,y_pred)

array([[491, 520],
       [508, 504]])

array([[508, 503],
       [491, 521]])